In [1]:
import wandb
import pandas as pd
import matplotlib.pyplot as plt

In [2]:

# Initialize wandb API
api = wandb.Api()

# Replace 'your_sweep_id' with your actual sweep ID
sweep_ids = ['1fli6j5p','rv3366qj']
project = 'maximes_other_crew/SeqBench_GSC/'


sweep_params = ['feat_size', 'model', 'test_dur']#'seed', 

In [3]:
sweep = api.sweep(project + sweep_ids[0])

In [4]:
sweep

<Sweep maximes_other_crew/SeqBench_GSC/1fli6j5p (FINISHED)>

In [5]:
runs = sweep.runs

In [6]:
runs

<Runs maximes_other_crew/SeqBench_GSC>

In [7]:
run = runs[0]

Exception: {'message': 'Cannot query field "internalId" on type "Project".', 'locations': [{'line': 3, 'column': 5}]}

In [21]:
for run in runs:
    print(3)

Exception: {'message': 'Cannot query field "internalId" on type "Project".', 'locations': [{'line': 3, 'column': 5}]}

In [15]:
# Initialize an empty DataFrame to hold the relevant data
data = []

for sweep_id in sweep_ids:

    # Fetch the sweep object
    sweep = api.sweep(project+sweep_id)

    # Fetch all runs in the sweep
    runs = sweep.runs
    # Extract the necessary data from each run
    for run in runs:
        config = run.config
        summary = run.summary
        feat_size = config.get('feat_size')
        model = config.get('model')
        # seed = config.get('seed')
        test_dur = config.get('test_dur')
        sched_factor = config.get('scheduler_factor')
        test_acc = summary.get('test_acc')

        if test_acc is not None:
            data.append({
                'feat_size': feat_size,
                'model': model,
                # 'seed': seed,
                'test_dur': test_dur,
                'test_acc': test_acc
            })



Exception: {'message': 'Cannot query field "internalId" on type "Project".', 'locations': [{'line': 3, 'column': 5}]}

In [5]:
data

[]

In [6]:
# Convert the data to a DataFrame for easier manipulation
df = pd.DataFrame(data)

# Calculate the number of runs for each line configuration
df['run_count'] = df.groupby(sweep_params).transform('size')

# Group by n_layers and lr, then compute the mean of best_valid_acc for each group
df_grouped = df.groupby(sweep_params+ ['run_count']).agg({'test_acc':['mean', 'std']}).reset_index()


df_grouped_sorted = df_grouped.sort_values(by=('test_acc','mean'), ascending=False).reset_index(drop=True)

KeyError: 'feat_size'

In [16]:
df_grouped_sorted[0:30]

feat_size        model test_dur run_count  test_acc          
                                                  mean       std
0        384          gru      300         3  0.957381  0.029387
1        384          gru      700         3  0.955403  0.029426
2        384          gru     1000         3  0.953039  0.029454
3        384          gru     2000         3  0.934047  0.027924
4        215          gru      300         3  0.919112  0.026878
5        215          gru      700         3  0.915001  0.027271
6        215          gru     1000         3  0.909818  0.027414
7        256        mamba      300         3  0.882213  0.040420
8        384          gru     3000         3  0.880358  0.019974
9        215          gru     2000         3  0.876879  0.027702
10       256  transformer      300         3  0.865940  0.032289
11       256        mamba      700         3  0.865260  0.040883
12       256        mamba     1000         3  0.837774  0.040581
13       128        mamba      300         3  0.818405  0.058565
14       215          gru     3000         3  0.805654  0.040253
15       128        mamba      700         3  0.804078  0.057806
16       128  transformer      300         3  0.781975  0.068452
17       128        mamba     1000         3  0.776801  0.056405
18       512           s4      300         3  0.737787  0.032789
19       256           s4      300         3  0.667891  0.037783
20       256        mamba     2000         3  0.624850  0.031983
21       128        mamba     2000         3  0.500926  0.177597
22       256        mamba     3000         3  0.370456  0.078641
23       128  transformer      700         3  0.310587  0.007179
24       512           s4      700         3  0.308459  0.035111
25       256           s4      700         3  0.294450  0.013796
26       128        mamba     3000         3  0.268238  0.146714
27       256  transformer      700         3  0.242392  0.061778
28       256           s4     1000         3  0.199670  0.015632
29       512           s4     1000         3  0.193033  0.011268

In [7]:
len(data)

0

In [8]:
df = df_grouped

df['model'] = df['model'].astype('category')
df['feat_size'] = df['feat_size'].astype('category')

# Plotting
plt.figure(figsize=(12, 8))

for model in df['model'].cat.categories:
    model_data = df[df['model'] == model]
    for size in model_data['feat_size'].cat.categories:
        size_data = model_data[model_data['feat_size'] == size]
        plt.errorbar(
            size_data['test_dur'],
            size_data['test_acc']['mean'],
            yerr=size_data['test_acc']['std'],
            label=f"{model} - {size}",
            fmt='-o'
        )

plt.xlabel('Test Duration')
plt.ylabel('Test Accuracy (mean ± std)')
plt.title('Test Accuracy over Test Duration for Different Models and Feature Sizes')
plt.legend(title='Model - Feature Size')
plt.grid(True)
plt.show()

NameError: name 'df_grouped' is not defined

In [9]:
df = df_grouped

df['model'] = df['model'].astype('category')
df['feat_size'] = df['feat_size'].astype('category')

# Plotting
plt.figure(figsize=(12, 8))

selected_pairs = [('gru', 215, 'GRU'), ('mamba', 128, 'Mamba'), ('transformer', 128, 'Transformer'), ('s4', 256, 'S4')]


for model, size, label in selected_pairs:
    filtered_data = df[(df['model'] == model) & (df['feat_size'] == size)]
    
    
    plt.plot(
        filtered_data['test_dur'],
        filtered_data['test_acc']['mean'],
        label=label,
        marker='o',
        linewidth=7,
        markersize=15
    )

fontsize = 20
# Increase font sizes
plt.xlabel('Test Gaps Duration (ms)', fontsize=fontsize)
plt.ylabel('Test Accuracy', fontsize=fontsize)
plt.title('Model Performance with respect to Gap size', fontsize=20)

# Increase legend font size
plt.legend(title='Model', title_fontsize=16, fontsize=14)

ticksize = 20
# Increase tick parameters
plt.xticks(fontsize=ticksize)
plt.yticks(fontsize=ticksize)

NameError: name 'df_grouped' is not defined

In [10]:
import itertools

# Define the grid sweep parameters
param_grid = {
    "lr": [0.01, 0.005, 0.001],
    "n_layers": [2, 3, 4, 5, 6, 7],
    "n_layers": [2, 3, 4, 5, 6, 7],
    "pdrop": [0.1, 0.25, 0.5],
}

# List of all launched runs (as a list of dictionaries containing the parameters used in each run)
launched_runs = data

# Generate all possible parameter combinations (the full grid)
all_combinations = list(itertools.product(
    param_grid['lr'], param_grid['n_layers'], param_grid['pdrop']
))

# Function to convert tuples to dictionaries for easy comparison
def tuple_to_dict(param_tuple):
    return {
        "lr": param_tuple[0],
        "n_layers": param_tuple[1],
        "pdrop": param_tuple[2],
    }

# Convert all combinations to dicts
all_combinations_as_dicts = [tuple_to_dict(combo) for combo in all_combinations]

# Extract relevant fields from launched runs
def extract_relevant_params(launched_run):
    return {
        "lr": launched_run['lr'],
        "n_layers": launched_run['n_layers'],
        "pdrop": launched_run['pdrop'],
    }

# Extract relevant parameters from launched runs for comparison
launched_params = [extract_relevant_params(run) for run in launched_runs]

# Find missing runs
missing_runs = [combo for combo in all_combinations_as_dicts if combo not in launched_params]


In [25]:
len(launched_params)

270

In [27]:
len(all_combinations_as_dicts)

72

In [24]:
len(missing_runs)

0

In [8]:
# Convert the data to a DataFrame for easier manipulation
df = pd.DataFrame(data)


df_grouped = df.groupby(sweep_params).agg({'best_valid_acc': ['mean', 'std']}).reset_index()

df_grouped_sorted = df_grouped.sort_values(by=('best_valid_acc','mean'), ascending=False).reset_index(drop=True)

In [70]:
df_grouped_sorted[0:10]

lr pdrop scheduler_patience scheduler_factor dt_max n_layers n_hiddens  \
                                                                               
0  0.005  0.25                 10              0.9    0.5        3       512   
1  0.010  0.10                 10              0.7    0.5        3       512   
2  0.010  0.10                  5              0.7    0.5        3       512   
3  0.005  0.25                  5              0.9    0.5        3       512   
4  0.005  0.50                  5              0.7    0.5        3       512   
5  0.005  0.25                 10              0.7    0.5        3       512   

  best_valid_acc            
            mean       std  
0       0.955437  0.003939  
1       0.952336  0.005383  
2       0.951081  0.005959  
3       0.949140  0.004174  
4       0.940664  0.008965  
5       0.939986       NaN